In [ ]:
import torchvision
from torchvision import datasets
from torchvision import transforms
import torch 
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, Subset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
from torchvision import models 

# to M1 GPU
torch.set_default_device('mps')
device = torch.device('mps')

transforms = transforms.Compose([transforms.ToTensor()])

# Load images
dataset = datasets.ImageFolder('/Users/alihodroj/Downloads/medicalMNIST', transform=transforms)


img_indices = list(range(len(dataset)))
train_indices, test_indices = train_test_split(img_indices, test_size=0.2, random_state=42)

train_dataloader = DataLoader(dataset, batch_size=100, sampler=SubsetRandomSampler(train_indices))
test_dataloader = DataLoader(dataset, batch_size=100, sampler=SubsetRandomSampler(test_indices))

model = models.resnet50(pretrained=True)
model

In [ ]:
for params in model.parameters():
    params.requires_grad = False 

In [ ]:

fc = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, 6),
            nn.LogSoftmax(dim=1)
        )
model.fc = fc 

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)


In [ ]:
# Train the model 
model.to(device)
epochs = 5
training_losses, test_losses = [], []

model.train()

for i in range(epochs): 
    print(f'Epoch {i} training a set of {len(train_dataloader)} batch_size={train_dataloader.batch_size}')
    running_loss = 0
    test_loss = 0 
    batch_num = 0

    for image, label in train_dataloader: 
        #print(f'-- Epoch {i}: batch # {batch_num} / {len(train_dataloader)}')    
        batch_num += 1
        
        # move to MPS
        image = image.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        y_pred = model(image)
        loss = criterion(y_pred, label)
        loss.backward() 
        optimizer.step()
        
        running_loss += loss.item()
    else: 
        with torch.no_grad(): 
            print(f'Epoch {i} evaluating a set of {len(test_dataloader)} batch_size={test_dataloader.batch_size}')
            model.eval()
            for t_images, t_labels in test_dataloader: 
            
                t_images = t_images.to(device) 
                t_labels = t_labels.to(device) 
            
                t_pred = model(t_images)
                test_loss += criterion(t_pred, t_labels)
                
                ps = torch.exp(t_pred)
                top_prediction, top_class = ps.topk(1, dim=1)
                equals = top_class == t_labels.view(*top_class.shape)
                accuracy = torch.mean(equals.type(torch.FloatTensor))
            
        # Put it back in training mode 
        model.train()

        training_losses.append(running_loss/len(train_dataloader))
        test_losses.append(test_loss/len(test_dataloader))
        print("Epoch: {}/{}.. ".format(i+1, epochs),
                "Training Loss: {:.3f}.. ".format(training_losses[-1]),
                "Test Loss: {:.3f}.. ".format(test_losses[-1]),
                "Test Accuracy: {:.3f}".format(accuracy/len(test_dataloader))) 


In [ ]:
plt.plot(training_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)